In [1]:
import glob
import os
import numpy as np
import pandas as pd
import zipfile

In [2]:
path = "Users\linathan\Documents\bts_downloads_2010_2023"
all_files = glob.glob(os.path.join(path , "\*.zip"))
li = []

In [3]:
path = os.getcwd()
path

'C:\\Users\\linathan\\Documents'

In [4]:
all_files = [path+"\\bts_downloads_2010_2023\On_Time_Reporting_Carrier_On_Time_Performance_1987_present_2010_1"]
name = "On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2010_1"

In [5]:
for filename in all_files:
    zf = zipfile.ZipFile(filename+'.zip')
    print(zf.namelist())
    df = pd.read_csv(zf.open(name+'.csv'))
    li.append(df)

['On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2010_1.csv', 'readme.html']


C:\Users\linathan\AppData\Local\Temp\1\ipykernel_16404\3870672093.py:4: DtypeWarning: Columns (77,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(zf.open(name+'.csv'))


In [6]:
## Extract input and output data as dataframe
X = li[0][['Reporting_Airline','Origin','Dest', 'ArrDelayMinutes']]
X = X.dropna(how = 'any')

In [7]:
## Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[['Reporting_Airline','Origin','Dest']], X[['ArrDelayMinutes']], 
                                                    shuffle=True, random_state=627, test_size=0.2)

In [8]:
## Create an instance of the dummy regressor, which always predicts the mean of the training data. i.e., the mean of delayed time
## The Dummy classifier is not meaningful, because most flighs are not cancelled or delayed.
## Thus, the dummy classifier will always predict that the flight is on time.
from sklearn.dummy import DummyRegressor
dummy_regr = DummyRegressor(strategy="mean")

In [9]:
#score records the cross validation mean accuracy
score = np.zeros((5))

In [21]:
##k-fold cross validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=626)

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit using Dummy Regressor
    dummy_regr.fit(X_train_train, y_train_train)
    
    ## Generate predictions on the holdout set
    dummy_pred = dummy_regr.predict(X_holdout)    

    ## Record mean accuracy of self.predict(X) w.r.t. y as a float
    score[i] = dummy_regr.score(X_holdout, y_holdout) 

In [22]:
score

array([-2.53914004e-06, -6.50027038e-07, -1.50723500e-09, -1.04439616e-05,
       -3.32961073e-05])

In [23]:
X = li[0][['Reporting_Airline','Origin','Dest', 'ArrDelayMinutes']]
X = X.dropna(how = 'any')
X=pd.get_dummies(X)
## Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, 1:], X[['ArrDelayMinutes']], 
                                                    shuffle=True, random_state=627, test_size=0.2)

In [24]:
## Create an instance
from sklearn.ensemble import RandomForestRegressor
forest_regr = RandomForestRegressor()

In [25]:
scoreForest = np.zeros((5))

In [26]:
##k-fold cross validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=626)

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit using Dummy Regressor
    forest_regr.fit(X_train_train, y_train_train)
    
    ## Generate predictions on the holdout set
    forest_pred = forest_regr.predict(X_holdout)    

    ## Record mean accuracy of self.predict(X) w.r.t. y as a float
    scoreForest[i] = forest_regr.score(X_holdout, y_holdout) 

C:\Users\linathan\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


KeyboardInterrupt: 

In [ ]:
scoreForest

In [28]:
X = li[0][['Reporting_Airline','Origin','Dest', 'ArrDelayMinutes']]
X = X.dropna(how = 'any')
X=pd.get_dummies(X)
## Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X.iloc[:, 1:], X[['ArrDelayMinutes']], 
                                                    shuffle=True, random_state=627, test_size=0.2)

from sklearn.linear_model import LinearRegression
linear_regr = LinearRegression()

scoreLinear = np.zeros((5))

##k-fold cross validation
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5,shuffle=True, random_state=626)

for i, (train_index, test_index) in enumerate(kfold.split(X_train, y_train)):
    ## get the kfold training data
    X_train_train = X_train.iloc[train_index, :]
    y_train_train = y_train.iloc[train_index]

    ## get the holdout data
    X_holdout = X_train.iloc[test_index, :]
    y_holdout = y_train.iloc[test_index]

    ## Fit using Dummy Regressor
    linear_regr.fit(X_train_train, y_train_train)
    
    ## Generate predictions on the holdout set
    linear_pred = linear_regr.predict(X_holdout)    

    ## Record mean accuracy of self.predict(X) w.r.t. y as a float
    scoreLinear[i] = linear_regr.score(X_holdout, y_holdout) 

scoreLinear

array([0.01632486, 0.01692929, 0.0184221 , 0.01739314, 0.01704852])